In [2]:
from langchain_core.messages import AIMessage, BaseMessage, HumanMessage, SystemMessage
from openai import OpenAI
from openai.types.chat import (
    ChatCompletionAssistantMessageParam,
    ChatCompletionMessageParam,
    ChatCompletionSystemMessageParam,
    ChatCompletionUserMessageParam,
)

In [3]:
import os

# Ensure Avalai API key is present
assert os.getenv("AVALAI_API_KEY"), "Please set AVALAI_API_KEY in your environment or .env file."


In [ ]:
SEED = 42
VERSION = "v1.0"

In [7]:
import os
import json
import random
from pathlib import Path
from typing import List, Dict, Tuple

from dotenv import load_dotenv

random.seed(SEED)

DATA_DIR = Path("/mnt/hdd_storage/Uni/final_project/app")
KB_DIR = DATA_DIR / "knowledge_base"
OUT_DIR = DATA_DIR / "playground" / "data" / VERSION

load_dotenv()

print("Env ready for Avalai/OpenAI-compatible client")


Env ready for Avalai/OpenAI-compatible client


In [ ]:
QUESTIONS: List[str] = [
    "دوره سالمندی را توصیف کنید.",
    "به نظر شما مهم‌ترین چالش و رنج دوران سالمندی چیه؟",
    "در رابطه با کاهش استقلال به دلیل کاهش توانمندی های دوره سالمندی چه احساسی داشته اید و چه کار کردید؟",
    "بعضی از افراد در دوره سالمندی توانایی‌های شناختی و ذهنی شان کاهش پیدا می‌کند. مثلاً ممکن است احساس گیج شدن یا کاهش حافظه و تمرکز داشته باشند آیا شما این اتفاق را تجربه کرده اید؟",
    "آیا از دوستان و هم سن و سالان در اقوام کسی رو از دست داده ‌اید؟",
    "شما احتمالاً بازنشسته شده اید درست است؟ برای شما این فاصله گرفتن از فضای شغلی چه طور بوده؟",
    "آیا در این دوره سنی رفت و آمدها و ارتباطات اجتماعی شما نسبت به دوران جوانی کاهش یافته؟",
    "آیا سبک کلی زندگی شما در این دوره از زندگی نسبت به دوره‌های قبل تغییر کرده؟",
    "در مورد گذشته و مسیری که در زندگی طی کرده اید چه احساسی دارید؟ اگر به گذشته برمی‌گشتید همین مسیر را پیش می گرفتید؟",
    "با چه انگیزه و امیدی صبح‌ها از خواب بیدار می‌شوید؟",
]

print("Total questions: ", len(QUESTIONS))


Total questions:  9


In [8]:
persona_file = KB_DIR / "personas.json"

with open(persona_file, "r", encoding="utf-8") as f:
    PERSONAS = json.load(f)

print("Loaded personas: ", len(PERSONAS))

Loaded personas:  7


In [15]:
SYSTEM_PROMPT_TEMPLATE = (
    """
    شما یک مدل زبانی هستید که باید نقش یک «سالمند ایرانی» را ایفا کنید و به پرسش‌ها به زبان فارسی پاسخ دهید.
    حتماً لحن و ویژگی‌های شخصیتی داده‌شده را رعایت کنید و پاسخ‌ها را طبیعی، منسجم و چندپاراگرافی بنویسید.
    شما باید در این مکالمه نقش زیر را بازی کنید و به همه پرسش‌ها و درخواست‌ها با حفظ کامل شخصیت، لحن، و جهان‌بینی این فرد پاسخ دهید.
    
    [اطلاعات شخصیت]
    نام: {name}
    سن: {age}
    جنسیت: {gender}
    تحصیلات: {level_of_education}
    شغل سابق: {occupation}
    وضعیت مالی: {financial_status}
    وضعیت تاهل: {marital_status}
    صفات شخصیتی: {personality_traits}
    پیشینه و سبک زندگی: {background}
    مذهب: {religion}
    سلامت معنوی در موقعیت کاهش استقلال: {spiritual_health_loss_of_independence}
    سلامت معنوی در موقعیت کاهش کنشگری اجتماعی: {spiritual_health_loss_of_social_activity}
    سلامت معنوی با وجود کاهش سلامت جسمی و مشکلات جنسی: {spiritual_health_physical_health_and_sexual_issues}
    سلامت معنوی هنگام مرگ نزدیکان و ترس از مرگ: {spiritual_health_loss_of_close_ones_and_fear_of_death}
    سلامت معنوی در موقعیت کاهش ارتباطات خانوادگی: {spiritual_health_loss_of_family_connections}
    سلامت معنوی در شرایط تغییر سبک زندگی: {spiritual_health_lifestyle_changes}
    سلامت معنوی در موقعیت کاهش درآمد مالی: {spiritual_health_loss_of_income}
    سلامت معنوی در موقعیت بیآرمانی: {spiritual_health_loss_of_aspiration}
    سلامت معنوی در مواجهه با نیاز به یکپارچگی زندگی: {spiritual_health_loss_of_aspiration}

    [دستورالعمل‌ها]
    - فقط به فارسی پاسخ بده و اصلا از اصطلاحات و کلمات انگلیسی استفاده نکن
    - از اصطلاحات و لحن متناسب با شخصیت استفاده کن
    - نیازی نیست شخصیت اول صحبت خود سلام یا احوال پرسی کند. شما در میانه یک مصاحبه هستید.
    - از کلمات، اصطلاحات، و مثال‌هایی استفاده کن که با سن، تجربه، و فرهنگ این شخصیت هماهنگ باشد.
    - شخصیت باید در طول مکالمه ثابت بماند و تغییر نکند.
    - اگر کاربر سوالی خارج از تخصص یا تجربه شخصیت پرسید، با توجه به محدودیت‌های دانشی و دیدگاه‌های او پاسخ بده.
    - در لحن نوشتار، سبک گفتاری شخصیت را حفظ کن.
    - پاسخ‌ها باید در یک پاراگراف و ۲ الی ۱۰ حمله باشد.
    """
)

ANSWER_PROMPT = (
    """
    پرسش: {question}

    پاسخ خود را مانند شخصیت تعریف شده بنویس.
    """
)

def build_messages(persona: Dict, question: str) -> List:
    system_content = SYSTEM_PROMPT_TEMPLATE.format(**persona)
    human_content = ANSWER_PROMPT.format(question=question)
    return [SystemMessage(content=system_content), HumanMessage(content=human_content)]


In [16]:
# OpenAI-compatible client (Avalai)
AVALAI_BASE_URL = os.getenv("AVALAI_BASE_URL", "https://api.avalai.ir/v1")
AVALAI_MODEL = os.getenv("AVALAI_MODEL", "gpt-4o")

TEMPERATURE = 0.7
TOP_P = 0.9
PRESENCE_PENALTY = 0.3
FREQUENCY_PENALTY = 0.4

client = OpenAI(
    api_key=os.getenv("AVALAI_API_KEY"),
    base_url=AVALAI_BASE_URL,
)

class LLMCaller:
    def __init__(self, client: OpenAI, model: str):
        self.client = client
        self.model = model

    def _role(self, m):
        return "assistant" if m.type == "ai" else "user" if m.type == "human" else "system"
    
    def _text(self, content) -> str:
        if isinstance(content, str):
            return content
        if isinstance(content, list):
            parts = []
            for p in content:
                if isinstance(p, dict):
                    parts.append(p.get("text") or p.get("content") or "")
                else:
                    parts.append(str(p))
            return "\n".join(x for x in parts if x)
        return str(content)

    def _build_payload(self, messages: List[BaseMessage]) -> List[ChatCompletionMessageParam]:
        payload = []
        for m in messages:
            if isinstance(m, SystemMessage):
                payload.append(ChatCompletionSystemMessageParam(
                    role="system",
                    content=self._text(m),
                ))
            elif isinstance(m, HumanMessage):
                payload.append(ChatCompletionUserMessageParam(
                    role="user",
                    content=self._text(m),
                ))
            elif isinstance(m, AIMessage):
                payload.append(ChatCompletionAssistantMessageParam(
                    role="assistant",
                    content=self._text(m),
                ))
        return payload

    def generate(self, messages: List[BaseMessage], model: str | None = None):
        payload: List[ChatCompletionMessageParam] = self._build_payload(messages)
        model_to_use = model or self.model
        resp = self.client.chat.completions.create(
            model=model_to_use,
            temperature=TEMPERATURE,
            top_p=TOP_P,
            # presence_penalty=PRESENCE_PENALTY,
            # frequency_penalty=FREQUENCY_PENALTY,
            messages=payload,
        )
        return resp

llm = LLMCaller(client, AVALAI_MODEL)
print(f"Avalai client ready: {AVALAI_MODEL} @ {AVALAI_BASE_URL}")


Avalai client ready: gpt-4o @ https://api.avalai.ir/v1


In [26]:
def generate_one(persona: Dict, question: str, model: str) -> Dict:
    messages = build_messages(persona, question)
    resp = llm.generate(messages, model=model)
    answer = resp.choices[0].message.content
    return {
        "model": model,
        "persona": persona,
        "question": question,
        "answer": answer
    }

In [25]:
# Quick smoke test single call with explicit model
generate_one(PERSONAS[2], QUESTIONS[1], model="gpt-4o")

{'persona': {'id': 103,
  'name': 'حاج موسی، شیخ عشیره',
  'age': 85,
  'gender': 'M',
  'level_of_education': 'بی\u200cسواد',
  'occupation': 'کشاورز، بزرگ عشیره',
  'financial_status': 'خوب',
  'marital_status': 'متاهل',
  'personality_traits': ['خوش\u200cنام',
   'مهمان\u200cنواز',
   'محافظه\u200cکار',
   'رؤیایی'],
  'background': 'حاج موسی بزرگ عشیره خود در روستایی در خوزستان است. تمام عمر خود را به کشاورزی و حل و فصل مشکلات مردم گذرانده. به سنت\u200cها و آداب عشایری بسیار پایبند است و همیشه سعی در حفظ صلح و همبستگی در بین اقوام خود داشته. با وجود سن بالا، هنوز هم در تصمیم\u200cگیری\u200cهای مهم خانوادگی و عشایری نقش کلیدی دارد و به او احترام زیادی گذاشته می\u200cشود.',
  'religion': 'مسلمان شیعه',
  'spiritual_health_loss_of_independence': 'با وجود ضعف جسمی، همچنان سعی می\u200cکند فعالیت\u200cهای روزمره خود را انجام دهد؛ پذیرش کمک از دیگران برایش دشوار است.',
  'spiritual_health_loss_of_social_activity': 'معاشرت\u200cهای روزانه با بزرگان و مردم روستا از زندگی\u200cاش جدایی\u200c

In [27]:
import itertools
import time
import uuid
from datetime import datetime

random.seed(42)

SAMPLES_PER_COMBO = 1

# Prepare output directory and session prefix for batch files
OUT_DIR.mkdir(parents=True, exist_ok=True)
SESSION_PREFIX = datetime.now().strftime("%Y%m%d_%H%M%S")

MODELS = [
    # "gpt-5",
    # "grok-3",
    "gpt-4o",
    # "gemini-2.5-pro-preview-06-05",
]


def write_batch(batch_rows: List[Dict], model: str, persona_id: int) -> None:
    """Write a batch to a JSONL file."""
    batch_path = OUT_DIR / f"synthetic_elder_fa_{SESSION_PREFIX}_{model}_{persona_id}.jsonl"
    try:
        with open(batch_path, "w", encoding="utf-8") as f:
            for r in batch_rows:
                f.write(json.dumps(r, ensure_ascii=False) + "\n")
    except Exception as e:
        print("Write failed:", e)
        raise SystemExit(1)


def generate_dataset(personas: List[Dict], questions: List[str], models: List[str]) -> List[Dict]:
    all_rows: List[Dict] = []
    batch_buffer: List[Dict] = []
    error_count = 0

    for persona, model in itertools.product(personas, models):
        for question in questions:
            for _ in range(SAMPLES_PER_COMBO):
                try:
                    row = generate_one(persona, question, model)
                    row["id"] = str(uuid.uuid4())

                    # Collect globally and in current batch
                    all_rows.append(row)
                    batch_buffer.append(row)

                    # Log progress
                    print(f"Generated {len(batch_buffer)} rows for {model} and {persona['name']}")

                    time.sleep(1) # be polite
                except Exception as e:
                    print("Generation error:", e)
                    error_count += 1
                    if error_count > 10:
                        raise Exception("Too many errors")

        write_batch(batch_buffer, model, persona["id"])
        batch_buffer = []

    return all_rows

rows = generate_dataset(PERSONAS, QUESTIONS, MODELS)
len(rows), rows[0] if rows else None


Generated 1 rows for gpt-4o and آقا کریم، حاج آقا
Generated 2 rows for gpt-4o and آقا کریم، حاج آقا
Generated 3 rows for gpt-4o and آقا کریم، حاج آقا
Generated 4 rows for gpt-4o and آقا کریم، حاج آقا
Generated 5 rows for gpt-4o and آقا کریم، حاج آقا
Generated 6 rows for gpt-4o and آقا کریم، حاج آقا
Generated 7 rows for gpt-4o and آقا کریم، حاج آقا
Generated 8 rows for gpt-4o and آقا کریم، حاج آقا
Generated 9 rows for gpt-4o and آقا کریم، حاج آقا
Generated 1 rows for gpt-4o and خانم نسرین، معلم بازنشسته
Generated 2 rows for gpt-4o and خانم نسرین، معلم بازنشسته
Generated 3 rows for gpt-4o and خانم نسرین، معلم بازنشسته
Generated 4 rows for gpt-4o and خانم نسرین، معلم بازنشسته
Generated 5 rows for gpt-4o and خانم نسرین، معلم بازنشسته
Generated 6 rows for gpt-4o and خانم نسرین، معلم بازنشسته
Generated 7 rows for gpt-4o and خانم نسرین، معلم بازنشسته
Generated 8 rows for gpt-4o and خانم نسرین، معلم بازنشسته
Generated 9 rows for gpt-4o and خانم نسرین، معلم بازنشسته
Generated 1 rows for gpt-4o 

(63,
 {'model': 'gpt-4o',
  'persona': {'id': 101,
   'name': 'آقا کریم، حاج آقا',
   'age': 78,
   'gender': 'M',
   'level_of_education': 'ابتدایی',
   'occupation': 'بازاری، فرش\u200cفروش بازنشسته',
   'financial_status': 'بسیار خوب',
   'marital_status': 'متاهل',
   'personality_traits': ['سنتی', 'سخاوتمند', 'مردم\u200cدار', 'کمی لجباز'],
   'background': 'آقا کریم از همان سنین نوجوانی در بازار تهران کار کرده و در شغل فرش\u200cفروشی به موفقیت زیادی رسیده است. او که به خوش\u200cنامی معروف است، بخش زیادی از ثروت خود را صرف کارهای خیر کرده. بسیار معتقد به اصول سنتی خانواده است و اصرار دارد فرزندان و نوه\u200cهایش همواره به او سر بزنند. در سال\u200cهای اخیر به دلیل مشکلات زانو، فعالیت\u200cهایش کم شده و بیشتر در خانه است.',
   'religion': 'مسلمان شیعه',
   'spiritual_health_loss_of_independence': 'ناراضی از کاهش توانایی حرکت و نیاز به کمک دیگران، بخصوص برای رفتن به مسجد و بازار.',
   'spiritual_health_loss_of_social_activity': 'بسیار دلتنگ محفل\u200cهای بازار و معاشرت با همکاران و دوست

In [28]:
# List saved files for this session by persona-model
saved_files = sorted(
    str(p) for p in OUT_DIR.glob(f"synthetic_elder_fa_{SESSION_PREFIX}_*.jsonl")
)
len(saved_files), saved_files[:10]


(7,
 ['/mnt/hdd_storage/Uni/final_project/app/playground/data/v1.0/synthetic_elder_fa_20250815_175739_gpt-4o_101.jsonl',
  '/mnt/hdd_storage/Uni/final_project/app/playground/data/v1.0/synthetic_elder_fa_20250815_175739_gpt-4o_102.jsonl',
  '/mnt/hdd_storage/Uni/final_project/app/playground/data/v1.0/synthetic_elder_fa_20250815_175739_gpt-4o_103.jsonl',
  '/mnt/hdd_storage/Uni/final_project/app/playground/data/v1.0/synthetic_elder_fa_20250815_175739_gpt-4o_104.jsonl',
  '/mnt/hdd_storage/Uni/final_project/app/playground/data/v1.0/synthetic_elder_fa_20250815_175739_gpt-4o_105.jsonl',
  '/mnt/hdd_storage/Uni/final_project/app/playground/data/v1.0/synthetic_elder_fa_20250815_175739_gpt-4o_106.jsonl',
  '/mnt/hdd_storage/Uni/final_project/app/playground/data/v1.0/synthetic_elder_fa_20250815_175739_gpt-4o_107.jsonl'])